In [ ]:
import glob
import json
import os
import numpy as np
import pandas as pd

from healthcareai_toolkit.clients import MedImageInsightClient
from healthcareai_toolkit import settings

In [ ]:
data_dir = settings.PARALLEL_TEST_DATA_ROOT
limit = 50
files = list(
    glob.glob(
        data_dir + "**/*.dcm",
        recursive=True,
    )
)
files = [f for f in files if os.path.isfile(f)]
print(len(files))

files = files[:limit]

print(len(files))

4833
50


In [ ]:
client = MedImageInsightClient(settings.MI2_MODEL_ENDPOINT)
submitter = client.create_submitter(return_as="generator_unordered")

Using ml_client base_url 1: https://management.azure.com
Using ml_client base_url 2: https://management.azure.com


In [ ]:
rel_from = data_dir
output_file = "embeddings.jsonl"
f = open(output_file, "w")
for index, result in submitter.submit(image_list=files, total=len(files)):
    path = os.path.relpath(files[index], rel_from)
    row = {
        "path": path,
        "test": path.startswith(("test")),
        "inlier": path.startswith(("ref", "test/inlier")),
        **result,
    }
    print(json.dumps(row), file=f)
f.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:04<00:00,  1.28s/it]


In [ ]:
with open(output_file, "r") as f:
    lines = [json.loads(line) for line in f.readlines()]

df = pd.DataFrame(lines)

df["StudyInstanceUID"] = df["path"].apply(lambda s: s.split(os.path.sep)[-3])
df["SeriesInstanceUID"] = df["path"].apply(lambda s: s.split(os.path.sep)[-2])
df["image_features"] = df["image_features"].apply(lambda x: np.array(x))

df.head()

,path,test,inlier,image_features,scaling_factor,StudyInstanceUID,SeriesInstanceUID
0,ref/CT/1.3.6.1.4.1.55648.212961843974261926001...,False,True,"[[-0.054619256407022476, 0.009249498136341572,...",4.513362,1.3.6.1.4.1.55648.2129618439742619260018430695...,1.3.6.1.4.1.55648.2129618439742619260018430695...
1,ref/CT/1.3.6.1.4.1.55648.212961843974261926001...,False,True,"[[-0.052052054554224014, 0.020784486085176468,...",4.513362,1.3.6.1.4.1.55648.2129618439742619260018430695...,1.3.6.1.4.1.55648.2129618439742619260018430695...
2,ref/CT/1.3.6.1.4.1.55648.212961843974261926001...,False,True,"[[-0.05330154299736023, 0.018595069646835327, ...",4.513362,1.3.6.1.4.1.55648.2129618439742619260018430695...,1.3.6.1.4.1.55648.2129618439742619260018430695...
3,ref/CT/1.3.6.1.4.1.55648.212961843974261926001...,False,True,"[[-0.05683630332350731, -0.014662034809589386,...",4.513362,1.3.6.1.4.1.55648.2129618439742619260018430695...,1.3.6.1.4.1.55648.2129618439742619260018430695...
4,ref/CT/1.3.6.1.4.1.55648.212961843974261926001...,False,True,"[[-0.06004421040415764, -0.02760297805070877, ...",4.513362,1.3.6.1.4.1.55648.2129618439742619260018430695...,1.3.6.1.4.1.55648.2129618439742619260018430695...
